# CyberSec Monitor 360 — Today analytics (Pandas/NumPy/Matplotlib)

This notebook demonstrates the **Jupyter + NumPy + Pandas + Matplotlib** part of the syllabus.


In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

DB_PATH = '../db.sqlite3'
conn = sqlite3.connect(DB_PATH)


In [ ]:
cases = pd.read_sql_query('SELECT id, created_at, severity, incident_type, status, verdict, title FROM soc_case', conn)
alerts = pd.read_sql_query('SELECT id, created_at, severity, incident_type, title FROM soc_alert', conn)

cases['created_at'] = pd.to_datetime(cases['created_at'])
alerts['created_at'] = pd.to_datetime(alerts['created_at'])

today = pd.Timestamp.now(tz=None).normalize()
cases_today = cases[cases['created_at'].dt.normalize() == today]
alerts_today = alerts[alerts['created_at'].dt.normalize() == today]

len(alerts_today), len(cases_today)


In [ ]:
# Severity percent (alerts)
sev_order = ['CRITICAL','HIGH','MEDIUM','LOW']
sev_counts = alerts_today['severity'].value_counts().reindex(sev_order, fill_value=0)
sev_percent = (sev_counts / max(len(alerts_today),1) * 100).round(1)
sev_percent


In [ ]:
# Plot: cases per hour
if not cases_today.empty:
    hourly = cases_today.set_index('created_at').resample('H').size()
    hourly.plot(kind='line')
    plt.title('Cases per hour (today)')
    plt.xlabel('Hour')
    plt.ylabel('Cases')
    plt.show()


In [ ]:
# Plot: incident types
if not alerts_today.empty:
    types = alerts_today['incident_type'].value_counts().head(10)
    types.plot(kind='bar')
    plt.title('Top incident types (today)')
    plt.xlabel('Type')
    plt.ylabel('Count')
    plt.show()
